
# Create a ReAct Agent

In this, we will create a virtual stock advisor agent that uses the **ReAct** prompting framework. This agent will reason and act based on the prompts and tools provided.

Of course, the agent is built for demonstration purposes and the answers shouldn't be used as investment advice!



In [0]:
%pip install -U -qq langchain-databricks langchain==0.3.7 langchain_community==0.3.5 wikipedia==1.4.0 youtube-search duckduckgo-search

dbutils.library.restartPython()

In [0]:
%run ../Includes/Classroom-Setup-03LAB

In [0]:
import mlflow
mlflow.langchain.autolog()

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

## Step 1: Define the Brain of the Agent

Let's start by defining the agent brain using one of the available LLMs on Databricks.

* For this lab, use **databricks-meta-llama-3-3-70b-instruct** 

* Define the maximum number of tokens

In [0]:
from langchain.agents import AgentType, initialize_agent
from langchain_databricks import ChatDatabricks

## define the brain
llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)

In [0]:
## let's test the brain
llm_llama.invoke("Hi! How are you?")

## Step 2 : Define Tools that the Agent Will Use

For an agent, the toolset is a crucial component to complete the defined task. The agent will reason about the next action to take and use an appropriate tool to complete that action.

For our stock advisor agent, we will use the following tools:

* **Wikipedia tool**: For searching company details. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/wikipedia/)]

* **DuckDuckGo Search tool**: This will be used for searching the latest news and updates about a stock.[[LangChain Doc](https://python.langchain.com/docs/integrations/tools/ddg/)]

* **YouTube search tool**: This will be used to search for review videos about the latest financial reports. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/youtube/)]

### Wikipedia Tool

In [0]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)
tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [0]:
## Test the Wikipedia tool
tool_wiki.run("Apple company history")

### DuckDuckGo Search Tool

In [0]:
## DuckDuckGo Search Tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

tool_news = DuckDuckGoSearchRun()

In [0]:
## Test DuckDuckGo tool
tool_news.run("Latest news about Apple stock")

### YouTube Search Tool

In [0]:
from langchain_community.tools import YouTubeSearchTool

tool_youtube = YouTubeSearchTool()

In [0]:
## Test the YouTube search tool 
tool_youtube.run("Apple company stock")

### Define Toolset

Next, add all tools together to be used in the next step.

In [0]:
tools = [tool_wiki, tool_youtube, tool_news]

## Step 3: Define Agent Logic

An agent requires a planning prompt that defines how it should **reason and act**. LangChain provides a few ready-to-use prompt templates through its **[LangChain Hub](https://smith.langchain.com/hub)**, including those that follow the **ReAct (Reason + Act)** style.

In this task, you'll pull a pre-defined ReAct-style prompt from LangChain Hub, which is designed to work with tool-using agents.

> 💡 Using the Hub simplifies your code and ensures you're working with prompt templates that are community-tested and regularly maintained.

📚 [Learn more: LangChain Agent Executor Documentation](https://python.langchain.com/docs/how_to/agent_executor/#create-the-agent)

In [0]:
## Import LangChain Hub
from langchain import hub

## Pull a ReAct-style prompt for function/tool-calling agents
prompt = hub.pull("hwchase17/openai-functions-agent")

## Display the prompt structure
prompt.messages

## Step 4: Create the Agent

The final step is to create the agent using the LLM, planning prompt and toolset.

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm_llama, tools, prompt)
agent_exec  = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Step 5: Run the Agent

Let's test the agent!

Note: YouTube search might not return the best results. For better results, you could use a paid API/Service that supports better search.

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

Format the final answer as HTML.
</blockquote>


In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

    Format the final answer as HTML.
    """})

displayHTML(response["output"])

It seems like the agent uses company name only when searching for videos. **Let's play with the prompt to get better results.**

In real-world, the agent would be used in a conversational way and we would just chat with the agent to tweak the search query or to adjust the response in a desired format. 

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

Write a nice summary of the results.

Format the final answer as HTML.
</blockquote>

In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.
    When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

    Write a nice summary of the results.

    Format the final answer as HTML.

    """})
displayHTML(response["output"])